In [0]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#scheme_code = 125497
url = f"https://api.mfapi.in/mf/latest"

response = requests.get(url)
data = response.json()
print(data)
df = spark.createDataFrame(data)
display(df)

In [0]:
df.count()

In [0]:
from datetime import date
import pandas as pd
today=date.today()
print(today)

In [0]:
df = df.toPandas()

csv_path = f"/Volumes/raw_data_files/mutual_funds/raw_files_daily_extraction/{today}_Nav_data.csv"
df.to_csv(csv_path, index=False)


In [0]:
source_path = "/Volumes/raw_data_files/mutual_funds/raw_files_daily_extraction"

df1 = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.schemaLocation", "/Volumes/raw_data_files/mutual_funds/schema")
        .option("header", "true")
        .option("cloudFiles.inferColumnTypes", "true")
        .load(source_path)
)


In [0]:
checkpoint_path = "/Volumes/raw_data_files/mutual_funds/checkpoint"
schema_location = "/Volumes/raw_data_files/mutual_funds/schema"

(
    df1.writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint_path)
        .option("cloudFiles.schemaLocation", schema_location)
        .option("mergeSchema", "true")
        .outputMode("append")
        .trigger(availableNow=True)
        .toTable("projects.bronze.mutual_funds_Data")
)